In [ ]:
#creating venv
!python -m venv env
!source ./env/bin/activate
#libs installation
!pip install -q pydub
!pip install -q sounddevice
!pip install -q rpunct
!pip install -q torch==1.7.0
!pip install -q torchaudio==0.7.0
!apt-get install -y libsndfile1 ffmpeg
!pip install -q Cython
!pip install -q -U nemo_toolkit[all]
!pip uninstall -q scikit-learn -y
!pip install -q scikit-learn==0.23.2
!pip install -q tslearn
!git clone https://github.com/NVIDIA/apex.git
!python3 ./apex/setup.py install
!pip install -q transformers[torch]
!pip install -q playsound pyaudio pydub ffmpeg-python

# command list
commands = ["activate",
            "bot",
            "story",
            "stop"]
numbers = {"one" : 1,
           "two" : 2,
           "three" : 3,
           "four" : 4,
           "five" : 5,
           "six" : 6,
           "seven" : 7,
           "eight" : 8,
           "nine" : 9}

probability_of_command = 0.5

fs = 16000  # Sample rate
seconds = 1  # Duration of recording

from difflib import SequenceMatcher
import torch
import soundfile as sf
from nemo.collections.tts.models.base import SpectrogramGenerator, Vocoder
from nemo.collections.asr.models import ASRModel, EncDecClassificationModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTNeoForCausalLM, GPT2Tokenizer, AutoModelWithLMHead, AutoTokenizer
from rpunct import RestorePuncts
from time import time
import sounddevice as sd
from scipy.io.wavfile import write
from pydub import AudioSegment
from playsound import playsound

def checkCommand(seq):
    prob = [SequenceMatcher(None, seq, seq2).ratio() for seq2 in commands+list(numbers.keys())]
    max_elem = max(prob)
    return [commands[i] for i, j in enumerate(prob) if j == max_elem] if max_elem>=probability_of_command else [""]

def recordVoice():
    !rm -R rec
    !mkdir rec
    
    !cp ../input/voiceexamples/empty.wav ./rec/main.wav
    !cp ../input/voiceexamples/empty.wav ./rec/trash.wav
    
    playsound('../input/voiceexamples/activation.wav')
    print("RECORDING")
    playsound('../input/voiceexamples/beep.wav')
    
    while not (torch.is_nonzero(activityDetector.transcribe(paths2audio_files=["./rec/trash.wav"], logprobs=False)[0])):
        myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=1)
        sd.wait()  # Wait until recording is finished
        write('./rec/trash.wav', fs, myrecording)  # Save as WAV file 
    while(torch.is_nonzero(activityDetector.transcribe(paths2audio_files=["./rec/trash.wav"], logprobs=False)[0])):
        main = AudioSegment.from_file("./rec/main.wav", format="wav")
        newActivity = AudioSegment.from_file("./rec/trash.wav", format="wav")
        combined = main + newActivity
        handle = combined.export("./rec/main.wav", format="wav")
        myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=1)
        sd.wait()  # Wait until recording is finished
        write('./rec/trash.wav', fs, myrecording)  # Save as WAV file 
        
    playsound('../input/voiceexamples/beep.wav')
        
generator = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").to("cuda")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

recognizer = ASRModel.from_pretrained(model_name="QuartzNet15x5Base-En").cuda().eval()

generator_voice = SpectrogramGenerator.from_pretrained("tts_en_fastpitch", strict=False).cuda().eval()
vocoder = Vocoder.from_pretrained("tts_hifigan", strict=False).cuda().eval()

bot_tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
bot = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large").to("cuda")

activityDetector = EncDecClassificationModel.from_pretrained(model_name="MarbleNet-3x2x64").cuda().eval()

#translator = AutoModelWithLMHead.from_pretrained("t5-base").to("cuda")
#translator_tokenizer = AutoTokenizer.from_pretrained("t5-base")

rpunct = RestorePuncts()

In [ ]:
recordVoice()
com = checkCommand(recognizer.transcribe(paths2audio_files=["./rec/main.wav"], logprobs=False)[0])
if "activate" in com:
    while(True):
        recordVoice()
        voice = checkCommand(recognizer.transcribe(paths2audio_files=["./rec/main.wav"], logprobs=False)[0])
        if "bot" in command:
            recordVoice()
            if x in list(numbers.keys()) for x in checkCommand(recognizer.transcribe(paths2audio_files=["./rec/main.wav"], logprobs=False)[0]):
                for step in range(numbers[x]):
                    recordVoice()
                    voice = rpunct.punctuate(recognizer.transcribe(paths2audio_files=inp_file, logprobs=False)[0])
                    new_user_input_ids = bot_tokenizer.encode(voice + bot_tokenizer.eos_token, return_tensors='pt').to("cuda")
                    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
                    chat_history_ids = bot.generate(
                        bot_input_ids,
                        max_length=1024,
                        #num_beams=5, # 3
                        #do_sample=True,
                        top_k=0, #100
                        temperature=0.75,
                        top_p=0.95,
                        early_stopping=True,
                        pad_token_id=tokenizer.eos_token_id)
                    answers = "Response: {}".format(bot_tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:].cpu()[0], skip_special_tokens=True))
                    audio = vocoder.convert_spectrogram_to_audio(
                                spec=generator_voice.generate_spectrogram(
                                    tokens=generator_voice.parse(answers)
                                )
                            )
                    sf.write("speech.wav", audio.cpu().detach().numpy()[0], 22050)
                    playsound('./speech.wav')
        elif "story" in command:
            recordVoice()
            voice = rpunct.punctuate(recognizer.transcribe(paths2audio_files=["./rec/main.wav"], logprobs=False)[0])
            generated = tokenizer.decode(generator.generate(tokenizer(voice, return_tensors="pt").input_ids.to("cuda"), do_sample=True, temperature=0.9, max_length=128).cpu()[0], skip_special_tokens=True)
            audio = vocoder.convert_spectrogram_to_audio(
                        spec=generator_voice.generate_spectrogram(
                            tokens=generator_voice.parse(answers)
                        )
                    )
            sf.write("speech.wav", audio.cpu().detach().numpy()[0], 22050)
            playsound('./speech.wav')
        elif "stop" in command:
            return;
        else:
            recordVoice()
            voice = recognizer.transcribe(paths2audio_files=["./rec/main.wav"], logprobs=False)[0]
else:
    recordVoice()
    com = checkCommand(recognizer.transcribe(paths2audio_files=["./rec/main.wav"], logprobs=False)[0])